<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-29 09:42:31
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.28 L
Current:  1.37 C
-------------------
Today PnL: 16.50 K (0.12%)
Current PnL: -20.69 L (-13.92%)
CY Booked + Current PnL: -11.65 L (-7.84%)
-------------------
Total profit:  2.70 L
Total loss:  -23.39 L
-------------------
Total Booked + Current PnL: 15.33 L (12.6%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.62%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 82.07 L (60.11%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.31%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.61,112392.0,8080.0,8564.0,0.15,7.75,7.62,15.96,9.0,0.94,0.85,25.95,X5K,ATH,METALS
53,MEDANTA,1486.00,12.80,59.0,H-MC,15.04,161089.0,34889.0,11292.0,-0.80,27.65,7.01,36.59,108.0,3.09,1.22,43.68,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,8285.00,-15.41,56.0,H-LC,8.41,169026.0,12849.0,13252.0,-0.58,8.23,7.84,16.71,22.0,0.97,1.28,27.10,X40N,BTT,HEALTHCARE
66,SBIN,863.00,-14.20,41.0,M-LC,4.25,212768.0,11288.0,15936.0,0.12,5.60,7.49,13.51,60.0,0.71,1.61,18.91,XY25,NTT,BANKS
81,TTKPRESTIG,770.00,97.01,55.0,M-SC,8.19,84901.0,-15876.0,15970.0,0.14,-15.75,18.81,0.09,245.0,-0.99,0.64,11.54,OX40N,NTT,DURABLES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RAJOOENG,143.33,-32.80,38.0,H-SC,9.99,101780.0,-720.0,41547.0,0.38,-0.70,40.82,39.83,114.0,-0.02,0.77,0.77,AR,ATH,MISC
39,ICICIPRULI,790.00,-20.77,37.0,H-MC,2.38,136334.0,546.0,42209.0,-1.03,0.40,30.96,31.48,107.0,0.01,1.03,12.57,X40,ATH,INSURANCE
38,ICICIGI,2260.25,-20.34,37.0,H-MC,4.91,137137.0,1004.0,30115.0,-0.38,0.74,21.96,22.86,91.0,0.03,1.04,13.44,X40,ATH,INSURANCE


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,7934.00,-42.98,30.0,H-LC,13.51,238416.0,-23203.0,142406.0,-0.67,-8.87,59.73,45.57,7.0,-0.16,1.80,1.45,AR,NTT,ELECTRICAL
33,HCLTECH,1943.91,-0.65,36.0,H-LC,10.24,227053.0,-14871.0,78152.0,-0.25,-6.15,34.42,26.15,8.0,-0.19,1.72,7.30,X40,ATH,IT
69,SIEMENS,7969.85,-5.82,40.0,H-LC,32.70,151420.0,-34675.0,247072.0,-0.49,-18.63,163.17,114.13,15.0,-0.14,1.14,11.51,AR,ATH,ELECTRICAL
47,JIOFIN,387.00,-1.23,40.0,H-LC,15.23,218470.0,1064.0,52433.0,0.64,0.49,24.00,24.61,48.0,0.02,1.65,55.58,XY24,BTT,FINANCE
84,UNITDSPR,1644.00,-14.94,41.0,H-LC,8.74,171006.0,-10753.0,46001.0,1.09,-5.92,26.90,19.39,86.0,-0.23,1.29,1.75,X40N,NTT,BREWERIES


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,NESTLEIND,1377.0,-11.02,55.0,H-LC,2.84,273967.0,8541.0,48246.0,0.72,3.22,17.61,21.40,11.0,0.18,2.07,10.78,XY25,NTT,FMCG
51,LTIM,7230.2,-7.23,47.0,H-LC,2.94,230175.0,-15976.0,95177.0,-0.16,-6.49,41.35,32.18,16.0,-0.17,1.74,27.21,X200,ATH,IT
45,ITC,452.0,-38.34,47.0,H-LC,3.01,199170.0,-968.0,21849.0,1.60,-0.48,10.97,10.44,4.0,-0.04,1.51,5.05,X40,NTT,FMCG
20,CIPLA,1795.0,-19.09,59.0,H-LC,4.37,215816.0,11316.0,30106.0,-0.32,5.53,13.95,20.25,10.0,0.38,1.63,14.75,X40N,BTT,PHARMA
5,ASIANPAINT,4250.0,-12.28,57.0,H-LC,7.23,221522.0,-30246.0,152474.0,1.14,-12.01,68.83,48.55,27.0,-0.20,1.67,19.16,X40,BTT,PAINTS


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,RELAXO,1176.00,-33.49,58.0,H-SC,8.48,61741.0,-53797.0,88790.0,7.74,-46.56,143.81,30.28,136.0,-0.61,0.47,21.38,X40N,NTT,FOOTWEAR
64,SAMMAANCAP,326.00,-164.77,52.0,M-SC,8.27,74166.0,-28044.0,121432.0,4.78,-27.44,163.73,91.37,208.0,-0.23,0.56,20.40,XY25,NTT,FINANCE
23,COLPAL,3767.14,-0.59,55.0,H-MC,10.64,180859.0,-30882.0,112983.0,2.48,-14.58,62.47,38.77,84.0,-0.27,1.37,7.65,XY25,ATH,FMCG
86,VALIANTORG,838.00,-311.48,43.0,H-SC,11.96,89934.0,-43352.0,127949.0,2.34,-32.53,142.27,63.47,139.0,-0.34,0.68,50.75,XR,NTT,CHEMICALS
15,BRITANNIA,6362.67,16.17,62.0,H-LC,8.87,210024.0,11928.0,19028.0,1.98,6.02,9.06,15.63,26.0,0.63,1.59,29.18,XY25,ATH,FMCG


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,EASEMYTRIP,26.40,-15.15,31.0,M-SC,29.69,81674.0,-95141.0,175934.0,-1.30,-53.81,215.41,45.70,197.0,-0.54,0.62,0.00,XY24,NTT,TRAVEL
42,INDUSINDBK,1800.00,246.64,33.0,L-MC,10.85,42556.0,-34650.0,60042.0,-1.30,-44.88,141.09,32.89,258.0,-0.58,0.32,17.20,XR,NTT,BANKS
87,VBL,671.64,-12.96,43.0,H-LC,8.06,313577.0,-2265.0,114926.0,-1.25,-0.72,36.65,35.67,5.0,-0.02,2.37,13.04,X40N,ATH,FMCG
67,SFL,1287.00,34.57,49.0,M-SC,9.44,177749.0,-84469.0,154304.0,-1.22,-32.21,86.81,26.63,229.0,-0.55,1.34,10.51,XY24,NTT,MISC
75,TANLA,1963.11,-29.36,46.0,H-SC,7.90,161533.0,-77882.0,350834.0,-1.21,-32.53,217.19,114.01,133.0,-0.22,1.22,48.15,AR,ATH,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,97.01,55.0,M-SC,8.19,84901.0,-15876.0,15970.0,0.14,-15.75,18.81,0.09,245.0,-0.99,0.64,11.54,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,80.86,39.0,H-SC,14.59,114264.0,-28215.0,47739.0,-0.26,-19.80,41.78,13.70,163.0,-0.59,0.86,34.86,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.70,50.0,H-SC,3.47,215721.0,-53946.0,90279.0,0.69,-20.00,41.85,13.47,138.0,-0.60,1.63,9.11,XY24,NTT,PAINTS
19,CERA,9475.0,-21.48,37.0,H-SC,3.06,111960.0,-31843.0,58589.0,-0.43,-22.14,52.33,18.60,149.0,-0.54,0.85,23.55,OX40N,NTT,CERAMICS
41,INDIGOPNTS,1408.0,107.03,38.0,M-SC,2.21,135644.0,-38915.0,38943.0,-0.31,-22.29,28.71,0.02,221.0,-1.00,1.03,17.99,OX40N,NTT,PAINTS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-27.08,47.0,H-LC,15.60,290451.0,-55505.0,179353.0,-0.12,-16.04,61.75,35.80,1.0,-0.31,2.20,2.89,X40,BTT,IT
43,INFY,2275.00,-18.88,47.0,H-LC,11.70,316561.0,3673.0,167999.0,-0.93,1.17,53.07,54.87,3.0,0.02,2.39,7.90,X40,BTT,IT
45,ITC,452.00,-38.34,47.0,H-LC,3.01,199170.0,-968.0,21849.0,1.60,-0.48,10.97,10.44,4.0,-0.04,1.51,5.05,X40,NTT,FMCG
87,VBL,671.64,-12.96,43.0,H-LC,8.06,313577.0,-2265.0,114926.0,-1.25,-0.72,36.65,35.67,5.0,-0.02,2.37,13.04,X40N,ATH,FMCG
1,ABB,7934.00,-42.98,30.0,H-LC,13.51,238416.0,-23203.0,142406.0,-0.67,-8.87,59.73,45.57,7.0,-0.16,1.80,1.45,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.0,12.80,59.0,H-MC,15.04,161089.0,34889.0,11292.0,-0.80,27.65,7.01,36.59,108.0,3.09,1.22,43.68,XY24,NTT,HEALTHCARE
27,DMART,5201.0,-7.27,73.0,H-LC,12.83,231623.0,47142.0,23232.0,0.58,25.55,10.03,38.14,38.0,2.03,1.75,38.87,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1486.00,12.80,59.0,H-MC,15.04,161089.0,34889.0,11292.0,-0.80,27.65,7.01,36.59,108.0,3.09,1.22,43.68,XY24,NTT,HEALTHCARE
14,BLUESTARCO,2326.38,9.32,59.0,H-MC,4.05,188700.0,24030.0,43929.0,0.37,14.59,23.28,41.28,89.0,0.55,1.43,23.74,X40N,ATH,AC
88,VOLTAS,1918.49,-2.76,56.0,H-MC,6.32,203940.0,12198.0,83840.0,0.02,6.36,41.11,50.08,99.0,0.15,1.54,13.59,XY25,ATH,AC
34,HEROMOTOCO,5949.07,-3.95,69.0,H-MC,15.85,177821.0,26908.0,30390.0,-0.22,17.83,17.09,37.97,93.0,0.89,1.34,47.65,AR,ATH,AUTO
24,DABUR,735.00,-6.90,53.0,H-MC,3.77,202664.0,5636.0,83984.0,1.13,2.86,41.44,45.49,102.0,0.07,1.53,16.27,XY24,BTT,FMCG


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,-3.95,69.0,H-MC,15.85,177821.0,26908.0,30390.0,-0.22,17.83,17.09,37.97,93.0,0.89,1.34,47.65,AR,ATH,AUTO
40,INDIAMART,4850.92,-50.91,50.0,H-SC,12.47,137318.0,13982.0,119782.0,0.51,11.34,87.23,108.45,119.0,0.12,1.04,36.22,AR,ATH,MISC
89,WHIRLPOOL,2270.00,-42.68,47.0,M-SC,5.67,96952.0,5454.0,73296.0,-0.30,5.96,75.60,86.07,223.0,0.07,0.73,40.79,XR,NTT,DURABLES
90,WIPRO,420.00,-11.07,54.0,M-LC,7.68,156302.0,5357.0,104097.0,0.58,3.55,66.60,72.51,53.0,0.05,1.18,9.79,XR,NTT,IT
59,RAJOOENG,143.33,-32.80,38.0,H-SC,9.99,101780.0,-720.0,41547.0,0.38,-0.70,40.82,39.83,114.0,-0.02,0.77,0.77,AR,ATH,MISC


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-45.67,63.0,L-SC,26.12,81792.0,-31757.0,71805.0,0.14,-27.97,87.79,35.27,268.0,-0.44,0.62,100.19,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-7.20,35.0,H-SC,16.50,86391.0,-14572.0,109103.0,-0.73,-14.43,126.29,93.63,148.0,-0.13,0.65,28.68,SR,ATH,CHEMICALS
52,MASFIN,398.61,-19.15,46.0,H-SC,12.71,91680.0,-6300.0,27907.0,-0.31,-6.43,30.44,22.05,152.0,-0.23,0.69,33.26,XR,ATH,FINANCE
35,HINDALCO,756.01,-18.25,56.0,H-LC,7.61,112392.0,8080.0,8564.0,0.15,7.75,7.62,15.96,9.0,0.94,0.85,25.95,X5K,ATH,METALS
82,UJJIVANSFB,60.00,80.86,39.0,H-SC,14.59,114264.0,-28215.0,47739.0,-0.26,-19.80,41.78,13.70,163.0,-0.59,0.86,34.86,OX40N,NTT,BANKS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,DMART,5201.00,-7.27,73.0,H-LC,12.83,231623.0,47142.0,23232.0,0.58,25.55,10.03,38.14,38.0,2.03,1.75,38.87,X40N,NTT,FMCG
34,HEROMOTOCO,5949.07,-3.95,69.0,H-MC,15.85,177821.0,26908.0,30390.0,-0.22,17.83,17.09,37.97,93.0,0.89,1.34,47.65,AR,ATH,AUTO
22,COFFEEDAY,80.00,-45.67,63.0,L-SC,26.12,81792.0,-31757.0,71805.0,0.14,-27.97,87.79,35.27,268.0,-0.44,0.62,100.19,XR,NTT,HOTELS
14,BLUESTARCO,2326.38,9.32,59.0,H-MC,4.05,188700.0,24030.0,43929.0,0.37,14.59,23.28,41.28,89.0,0.55,1.43,23.74,X40N,ATH,AC
72,STARHEALTH,761.00,10.00,59.0,H-SC,2.55,163350.0,-36866.0,112891.0,1.64,-18.41,69.11,37.97,144.0,-0.33,1.23,31.16,XY24,NTT,INSURANCE


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.68
1,25,43.12
2,50,72.74


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    36.97
LC    36.81
MC    26.18
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.17
X40      15.52
X40N     14.28
XY25      9.79
AR        9.28
XR        8.92
OX40N     7.29
X5K       2.24
X200      1.74
MH        1.66
SR        1.07
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.65
H-MC    23.31
H-SC    22.34
M-SC    13.10
M-LC     7.11
M-MC     2.55
L-SC     1.53
L-LC     1.05
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.24,-1.31,34.31
IT,13.38,-16.67,74.53
BANKS,7.33,-17.49,67.38
FINANCE,7.22,-26.10,72.60
PAINTS,5.94,-17.40,43.67
MISC,5.82,-19.37,87.80
ELECTRICAL,5.62,-10.64,71.62
AUTO,4.53,-12.67,61.69
AC,3.70,1.07,32.44


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2909220.0,23
AR,1251199.0,10
XR,1111803.0,13
X40,915124.0,10
X40N,545360.0,11
OX40N,493740.0,10
XY25,446866.0,7
SR,200912.0,2
X5K,142685.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2818807.0,27
M-SC,1512514.0,17
H-LC,1440270.0,17
H-MC,1293235.0,18
M-LC,438254.0,5
M-MC,339910.0,2
L-SC,258884.0,3
L-MC,60042.0,1
L-LC,45401.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,947692.0,6
M-SC,XY24,887505.0,7
H-SC,AR,670118.0,5
H-LC,X40,599827.0,5
H-SC,XR,533075.0,6
H-MC,XY24,497580.0,5
H-LC,AR,389478.0,2
M-MC,XY24,339910.0,2
H-MC,X40,231576.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
